Avaliação - Unidade 02
===

Identificação dos alunos (nome completo): 

- Este trabalho irá compor 40% da segunda unidade.
- Grupo de até dois alunos.
- Os alunos deverão enviar (apenas um por grupo) através do Sigaa, o notebook e os datasets gerados na avaliação em um arquivo compactado (.zip). 
- Todas as respostas devem ser realizadas através de código python e os resultados explicados em estilo markdown
- As informações (datasets) necessárias para resolução dos problemas podem ser encontradas em http://dados.ufrn.br/group/ensino


1. (1 ponto) Quantas e quais disciplinas com código IMD foram consolidadas em 2017.1? 
**Dica:** 
>```python
df[df['A'].str.contains("hello")]
```
2. (3 pontos) Realize um estudo envolvendo as propriedades de **assimétria** e **curtose** para as turmas IMD0033 (probabilidade) consolidadas em 2017.1. Utilize gráficos e explique os resultados. Reforce também os horários das disciplinas nos resultados. 
3. (3 pontos) Utilize a biblioteca seaborn para traçar gráficos que descrevam as distribuições estatísticas (utilizar as funções **facetgrid** e **map**) de todas as turmas IMD0030 (Linguagem de Programação I) consolidas em 2017.1. Na análise, assuma como referência as médias finais e a situação dos alunos. 
4. (3 pontos) Utilize a biblioteca seaborn para traçar gráficos de histograma referente as médias finais de todas as turmas IMD0029 (Estrutura de dados I) consolidadas em 2017.1. Incremente nos gráficos linhas que descrevam a média e mediana da distribuição. 

Importação e preparação dos dados
==

In [ ]:
import pandas as pd


turmas_url = 'http://bit.do/turmas2017_1'
notas_url = 'http://bit.do/notas2017_1'

# turmas 2017.1
turmas = pd.read_csv(turmas_url, sep=';')

# matriculas em componentes
notas = pd.read_csv(notas_url, sep=';')

In [ ]:
# verificando os dados das turmas
print(turmas.columns)
turmas.head()

In [ ]:
# verificando os dados dos componentes curriculares
print(notas.columns)
notas.head()

In [ ]:
# filtrando as turmas com código IMD
turmas_imd = turmas[turmas.codigo.str.contains("IMD")]

# utilizar apenas disciplinas com código "CONSOLIDADA"
turmas_imd = turmas_imd[turmas_imd.situacao_turma == 'CONSOLIDADA']

# resetar os idexes
turmas_imd.reset_index(drop=True, inplace=True)

# visualizar o novo dataset
turmas_imd.head()

### 1. (1 ponto) Quantas e quais disciplinas com código IMD foram consolidadas em 2017.1?


In [ ]:
# Quantas turmas?
print('Total de turmas com código IMD consolidadas em 2017.1: %d' % (len(turmas_imd.id_turma.unique())))

# Quais turmas (limitei os nomes repetidos)
print(turmas_imd.nome_ascii.unique())

### 2. (3 pontos) Realize um estudo envolvendo as propriedades de assimétria e curtose para as turmas IMD0033 (probabilidade) consolidadas em 2017.1. Utilize gráficos e explique os resultados. Reforce também os horários das disciplinas nos resultados.

In [ ]:
# Fazendo um merge entre o dataframe turmas_imd e notas utilizando como base a coluna id_coluna
# https://pandas.pydata.org/pandas-docs/stable/merging.html
merge_imd = turmas_imd.merge(notas, on="id_turma", how='outer')

# Filtrar apenas as colunas que fazem sentido para o problema
merge_imd = merge_imd[['codigo','nome_ascii','descricao_horario','id_discente','media_final','situacao']]

# Apresentar o novo dataset
merge_imd.head()

In [ ]:
def dates_form_especific_disc(all_disc, code):
    # Filtrar a disciplina IMD0030
    disc = all_disc[all_disc['codigo'] == code]

    # Eliminar as linhas redundantes dos discentes
    disc = disc.drop_duplicates(subset='id_discente')

    # Eliminar os alunos com situação 'CANCELADO'
    disc = disc[disc.situacao != 'CANCELADO']

    # Verificar a existência de valores nulos
    disc = disc[~disc['media_final'].isnull()]
    
    # Eliminate informação desnecessária 
    # 24M12 (13/02/2017 - 01/07/2017) -> 24M12
    disc.descricao_horario = disc.descricao_horario.apply(lambda x: x[:5])
    
    return disc

In [ ]:
imd0033 = dates_form_especific_disc(merge_imd, "IMD0033")

# Visualizar o novo dataset
imd0033.head()

In [ ]:
from scipy.stats import skew,kurtosis

skewness = []
kurtosiness = []

# Calcular o skew e kurtosis para cada turma (horários diferentes)
for horario in imd0033.descricao_horario.unique():
    skewness.append(skew(imd0033[imd0033['descricao_horario'] == horario]['media_final']))
    kurtosiness.append(kurtosis(imd0033[imd0033['descricao_horario'] == horario]['media_final']))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Criando a figura
fig, (ax1,ax2) = plt.subplots(figsize=(10,5), ncols=2, nrows=1)

# Lista de eixos
eixos = [ax1, ax2]

# Lista de amostras
amostras = []

# Configurar os espaços entre as figuras
fig.tight_layout()

# Título da figura
plt.suptitle("IMD003[2017.1] - Análise de assimetria e curtose", 
             fontsize=20,
            y = 1.09)

# Margem do título
y_title_margin = 1

for i,horario in enumerate(imd0033.descricao_horario.unique()):
    # Títulos dos subplots
    eixos[i].set_title("%s skew = %.4f | kurtosis= %.4f "% (horario,skewness[i],kurtosiness[i]), 
              y = y_title_margin, fontsize=12)
    amostras.append(imd0033[imd0033['descricao_horario'] == horario]['media_final'])

# Gerando as distribuições com o Seaborn    
sns.distplot(amostras[0], kde = False, ax=ax1)
sns.distplot(amostras[1], kde = False, ax=ax2)

plt.show()

### (3 pontos) Utilize a biblioteca seaborn para traçar gráficos que descrevam as distribuições estatísticas (utilizar as funções facetgrid e map) de todas as turmas IMD0030 (Linguagem de Programação I) consolidas em 2017.1. Na análise, assuma como referência as médias finais e a situação dos alunos.

In [ ]:
# Filtrar a disciplina IMD0030
imd0030 = dates_form_especific_disc(merge_imd, "IMD0030")

# Visualizar o novo dataset
imd0030.head()

In [ ]:
imd0030.media_final = pd.to_numeric(imd0030['media_final'])

In [ ]:
imd0030.info()

In [ ]:
import numpy as np
np.seterr(divide='ignore', invalid='ignore')

# Colunas serão os horários, as linhas a situação do aluno
g = sns.FacetGrid(imd0030, col="descricao_horario", hue="situacao", size=3)
# For each subset of values, generate a kernel density plot of the "situacao" columns.
g.map(sns.kdeplot, "media_final", shade=True)

g.add_legend()

g.set(ylim=(0, 6));

sns.despine(left=True, bottom=True)

plt.suptitle("Estimativa de densidade Kernel em relação à situação e a média final dos alunos de LP1", 
             fontsize=15,
            y = 1.09)

plt.show()


# Plot the graph
plt.show()

### 4. (3 pontos) Utilize a biblioteca seaborn para traçar gráficos de histograma referente as médias finais de todas as turmas IMD0029 (Estrutura de dados I) consolidadas em 2017.1. Incremente nos gráficos linhas que descrevam a média e mediana da distribuição.

In [ ]:
# Filtrar a disciplina IMD0029
imd0029 = dates_form_especific_disc(merge_imd, "IMD0029")

# Visualizar o novo dataset
imd0029.head()

In [ ]:
def vertical_mean_line(x,**kwargs):
    plt.axvline(x.mean(),color="g")
    
def vertical_median_line(x,**kwargs):
    plt.axvline(x.median(),color="r")

In [ ]:
# Colunas serão os horários, as linhas a situação do aluno
g = sns.FacetGrid(imd0030, col="descricao_horario",size=6)

# O eixo x será as notas dos alunos
g.map(plt.hist, "media_final")

# Plotando linhas verticais com a média (verde) e mediana (vermelho) 
g.map(vertical_mean_line, "media_final")
g.map(vertical_median_line, "media_final")


# Plot the graph
plt.show()